# Series

Series' data can be many different things:

- a Python dict
- an `ndarray`
- a scalar value (like 2020)

A Pandas Series is like a column in a table: it is a one-dimensional array holding data of any type.

So, in essence
- **Series is a one-dimensional labeled array** capable of holding any data type (integers, strings, floating point numbers, Python objects, etc.). 
- The **axis labels** are collectively referred to as the index. 



![images/01_table_series.svg](images/01_table_series.svg)


To create a series:

```s = pd.Series(data, index=index)```

More simply, the basic method to create a `Series` is to call:

In [ ]:
# sets the backend of matplotlib to the 'inline' backend:
# With this backend, the output of plotting commands is displayed inline within frontends like the Jupyter notebook, directly below the code cell  
# that produced it. The resulting plots will then also be stored in the notebook document.
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
s = pd.Series(dtype='float')
s

## Make a Serie
### From ` ndarray `
- If data is an `ndarray`, index must be the same length as data. 
- If no index is passed, one will be created having values [0, ..., len(data) - 1].

In [ ]:
v = np.random.normal(size=5)
type(v)

In [ ]:
s = pd.Series(v, index=tuple('abcde'))
s

The index is given by

In [ ]:
s.index

The np.array of values is given by 

In [ ]:
s.values

And, **values can be accessed by label index**

In [ ]:
s.a

In [ ]:
s['a']

In [ ]:
s.loc['a']

or 0-based index (be careful!)

In [ ]:
s.iloc[0]

In [ ]:
s[0]

It is also possible to do **slicing** (We'll see more of it latter)

In [ ]:
s['b':'d']

### From dict

Series can be instantiated from dictionaries.

When the data is a dict, and an index is not passed, the Series index will be ordered by the dict’s insertion order (Python >= 3.6 and Pandas >= 0.23).

In [ ]:
d = {k: v for k, v in zip('abcde', np.random.normal(size=5))}
d

In [ ]:
s = pd.Series(d)
s

### From Scalar

If data is a scalar value, an index must be provided. The value will be repeated to match the length of index.

In [ ]:
s = pd.Series(1.0, index=tuple('olivia'))
s

But be careful, indexes are not necessarily unique. 

In [ ]:
s['i']

- when index is unique, pandas use a hashtable to map key to value: O(1)
- when index is non-unique and sorted, pandas use binary search: O(logN)
- when index is random ordered pandas need to check all the keys in the index: O(N)

Let us then check the access time for these cases:

In [ ]:
idx_unique = 'defghabcijkqrstuvwlmnopxyz'

print('index length', len(idx_unique), ' (unique: ', len(set(idx_unique)), ')')

s = pd.Series(1.0, index=tuple(idx_unique))

print("unique & unsorted")
%timeit s['z']

print("unique & sorted")
s.sort_index(inplace=True)
%timeit s['z']

In [ ]:
idx_not_unique = "oliviaoliviaoliviaoliviaol"
print('index length', len(idx_not_unique), ' (unique: ', len(set(idx_not_unique)), ')')

s = pd.Series(1.0, index=tuple(idx_not_unique))

print("not unique & unsorted")
%timeit s['a']

print("not unique & sorted")
s.sort_index(inplace=True)
%timeit s['a']

## NaN
`NaN` (*not a number*) is the standard missing data marker used in pandas.


In [ ]:
d = {'a' : 1, 'b' : 2, 'c': 3}

pd.Series(d, index=tuple('abz'))

## Get series values
The type of the values can be obtained via `dtype`

In [ ]:
s

In [ ]:
# as of pandas 0.24.0, the default dtype is 'float64'
s.dtype

And the values themselve

In [ ]:
# array of values as a numpy array 
print(type(s.values))
s.values

In [ ]:
# as a PandasArray
print(type(s.array))
s.array

In [ ]:
# also as a numpy array
print(type(s.array))
s.to_numpy()

## Slicing
Series acts very similarly to a ndarray, and is a valid argument to most NumPy functions. 
However, operations such as slicing will also slice the index.

In [ ]:
s = pd.Series(np.random.normal(size=5), index=tuple('abcde'))
s

Get the indeces that satisfy some filter

In [ ]:
s>0

Given the filter, you can get the values

In [ ]:
s[s>0]

Another example

In [ ]:
s[s>s.median()]

To get values given the indeces (using the 0-based indexing, I always prefer to use `iloc` to avoid any confusion)

In [ ]:
s.iloc[[0, 1, 2]]

In [ ]:
s[['a', 'c', 'e']]

To change the series' values you can use the same indexing access

In [ ]:
s["a"] = 10
s

In [ ]:
s.loc["b"] = 20
s

In [ ]:
s.iloc[2:] = 30
s

In [ ]:
s.iloc[-1] = 40
s

## Exercise
Before running them, try to guess the output of the followiong lines 

In [ ]:
s1 = pd.Series(np.random.randn(10), index=list(range(0, 50, 5)))
s1

In [ ]:
s1[5]

In [ ]:
s1.loc[5]

In [ ]:
s1.loc[:5]

In [ ]:
# !
s1[:5]

In [ ]:
s1.iloc[:5]

In [ ]:
s1.iloc[5]

## Operations
When working with raw NumPy arrays, looping through value-by-value is usually not necessary. The same is true when working with Series in pandas. Series can also be passed into most NumPy methods expecting an ndarray.

In [ ]:
r = pd.Series([1, 2, 3, 4, 5], index=tuple('abcde'))
s = pd.Series([1, 2, 3, 4, 5], index=tuple('edcba'))

In [ ]:
r + s

In [ ]:
2 * r

In [ ]:
r ** 2

In [ ]:
2 ** r

The result of an operation between unaligned Series will have the union of the indexes involved. If a label is not found in one Series or the other, the result will be marked as missing NaN. 

In [ ]:
s[1:] + s [:-1]

similar

In [ ]:
s.iloc[1:] + s.iloc[:-1]

## Describe & Visualize data

In [ ]:
s = pd.Series(np.random.normal(size=10000))
s.head(10)

In [ ]:
s.info()

In [ ]:
s.describe()

You can do several kinds of plot such as line, bar, box, etc. (see https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DataFrame.plot.html)

In [ ]:
s.plot(figsize=(25, 10))

In [ ]:
s.plot(kind='box')

In [ ]:
s.plot(kind='hist', bins=50)